In [1]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [32]:
env = GridworldEnv()

{0: [(1.0, 0, 0.0, True)], 1: [(1.0, 0, 0.0, True)], 2: [(1.0, 0, 0.0, True)], 3: [(1.0, 0, 0.0, True)]}


In [51]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        # TODO: Implement!
        #Vnext is V_k+1
        Vnext = np.zeros(env.nS)
        for s in range(env.nS):
            vnext = 0
            P = env.P[s]
            for a in range(env.nA):
                prob, next_s, reward, is_done = P[a][0]
                successors_sum = sum([successor[2]+discount_factor*V[successor[1]]*successor[0] for successor in P[a]])
                vnext += policy[s][a] * successors_sum
            Vnext[s] = vnext
        is_theta = (V-Vnext) < theta
        if is_theta.all():
            break
        V = np.copy(Vnext)
    return np.array(V)

In [52]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
print (random_policy)
v = policy_eval(random_policy, env)

[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
[  0.         -13.99988715 -19.99983277 -21.99981286 -13.99988715
 -17.99985268 -19.99983389 -19.99983277 -19.99983277 -19.99983389
 -17.99985268 -13.99988715 -21.99981286 -19.99983277 -13.99988715
   0.        ]


In [50]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

We implemented David Silver's algorithm, whose notation suffers from some "incompleteness". For example, the reward is $R_s^a$ but it's clear that the reward depends on which successor state we end up at, so it should be $R_{ss_2}^a$.